In [ ]:
# If not installed, run this cell. It is needed for access to GEE
!pip install ee

In [5]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='ee-jvg')

In [ ]:
# If not installed, run this cell. It helps visualization of data
!pip install geemap
!pip show geemap

In [16]:
import geemap
Map = geemap.Map()
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [23]:
# Access an image asset from my account
guate_bdry = ee.FeatureCollection('users/jvg/guatemala_buffer')
guate_geom = guate_bdry.geometry();
Map.centerObject(guate_geom);
Map.addLayer(guate_bdry, {"color": 'red'}, 'Guatemala Boundary');

In [ ]:
clay = ee.Image('OpenLandMap/SOL/SOL_CLAY-WFRACTION_USDA-3A1A1A_M/v02')
clay_clipped = clay.clip(guate_bdry)
Map.addLayer(clay_clipped, {}, 'clay_clipped')

In [6]:
# Let's import other libraries
import pandas as pd